In [1]:
# Medicare Part D: Generic vs Brand Name Savings Analysis
# Case Study: How much could Medicare save by increasing generic drug adoption?

import pandas as pd
import requests
import numpy as np
import sqlite3
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

print("MEDICARE PART D: GENERIC vs BRAND NAME SAVINGS ANALYSIS")
print("=" * 80)
print(f"Analysis Date: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print("Data Source: CMS Medicare Part D Spending by Drug API")
print("API Endpoint: https://data.cms.gov/data-api/v1/dataset/7e0b4365-fd63-4a29-8f5e-e0ac9f66a81b/data")
print("Business Question: How much could Medicare save with generic adoption?")
print("=" * 80)

MEDICARE PART D: GENERIC vs BRAND NAME SAVINGS ANALYSIS
Analysis Date: 2025-08-17 16:42:34
Data Source: CMS Medicare Part D Spending by Drug API
API Endpoint: https://data.cms.gov/data-api/v1/dataset/7e0b4365-fd63-4a29-8f5e-e0ac9f66a81b/data
Business Question: How much could Medicare save with generic adoption?


In [2]:
# Functions to fetch data

def fetch_medicare_part_d_data(limit=5000, offset=0):
    """
    Fetch Medicare Part D data from CMS API
    
    Parameters:
    limit (int): Number of records to fetch (API appears to max at 5000)
    offset (int): Starting record number for pagination
    
    Returns:
    pandas.DataFrame: Medicare Part D drug spending data
    """
    
    base_url = "https://data.cms.gov/data-api/v1/dataset/7e0b4365-fd63-4a29-8f5e-e0ac9f66a81b/data"
    
    # Try different parameter combinations that might work with this API
    params = {
        'size': limit,
        'offset': offset
    }
    
    # Alternative parameter names to try if the above doesn't work
    alt_params = [
        {'limit': limit, 'skip': offset},
        {'$limit': limit, '$offset': offset},
        {'per_page': limit, 'page': offset // limit + 1}
    ]
    
    try:
        print(f"Fetching Medicare Part D data (limit={limit}, offset={offset})...")
        
        # Try main parameters first
        response = requests.get(base_url, params=params, timeout=30)
        response.raise_for_status()
        
        data = response.json()
        df = pd.DataFrame(data)
        
        print(f"Successfully fetched {len(df)} records")
        
        # If we got fewer records than expected, let's check the response headers for pagination info
        if len(df) < limit and offset == 0:
            print(f"Response headers (for debugging):")
            for key, value in response.headers.items():
                if any(word in key.lower() for word in ['total', 'count', 'page', 'limit']):
                    print(f"      {key}: {value}")
        
        return df
        
    except requests.exceptions.RequestException as e:
        print(f"Error fetching data: {e}")
        
        # Try alternative parameter combinations
        for i, alt_param in enumerate(alt_params):
            try:
                print(f"   Trying alternative parameter set {i+1}: {alt_param}")
                response = requests.get(base_url, params=alt_param, timeout=30)
                response.raise_for_status()
                data = response.json()
                df = pd.DataFrame(data)
                print(f"   Alternative method worked! Fetched {len(df)} records")
                return df
            except:
                continue
        
        print(f"   All parameter combinations failed")
        return pd.DataFrame()

def fetch_all_medicare_data(batch_size=5000, target_records=14309):
    """Fetch all available Medicare Part D data using proper pagination"""
    
    all_data = []
    offset = 0
    
    print(f"Fetching Medicare Part D data (target: {target_records:,} records)...")
    print(f"   API appears to limit responses to {batch_size:,} records per request")
    
    while True:
        batch = fetch_medicare_part_d_data(limit=batch_size, offset=offset)
        
        if batch.empty:
            print(f"   No more data returned at offset {offset:,}")
            break
            
        all_data.append(batch)
        
        # Calculate running total
        total_so_far = sum(len(df) for df in all_data)
        print(f"   Batch {len(all_data)}: {len(batch):,} records | Total: {total_so_far:,}")
        
        # If we got less than batch_size, we've reached the end
        if len(batch) < batch_size:
            print(f"   Reached end of data (received {len(batch):,} < {batch_size:,})")
            break
            
        # If we've reached our target, we can stop (optional safety check)
        if total_so_far >= target_records:
            print(f"   Reached target of {target_records:,} records")
            break
        
        # Increment offset for next batch
        offset += len(batch)
        
        # Add a small delay to be respectful to the API
        import time
        time.sleep(0.1)
    
    if all_data:
        combined_df = pd.concat(all_data, ignore_index=True)
        expected_vs_actual = f"Expected: {target_records:,} | Actual: {len(combined_df):,}"
        
        if len(combined_df) == target_records:
            print(f"SUCCESS: Fetched all {len(combined_df):,} records!")
        elif len(combined_df) < target_records:
            print(f"PARTIAL: Fetched {len(combined_df):,} of {target_records:,} records")
            print(f"   Possible reasons: API limits, data changes, or network issues")
        else:
            print(f"UNEXPECTED: Fetched {len(combined_df):,} records (more than expected {target_records:,})")
        
        print(f"   {expected_vs_actual}")
        return combined_df
    else:
        print("No data retrieved from any batch")
        return pd.DataFrame()


In [3]:
# Extract Data from CMS API

print("\nSTEP 1: Extracting All Medicare Part D Data from CMS API")
df = fetch_all_medicare_data(batch_size=5000, target_records=14309)

# Display basic info about the dataset
if not df.empty:
    print(f"\nDataset Overview:")
    print(f"   Total Drug Records: {len(df):,}")
    print(f"   Columns: {list(df.columns)}")
    print(f"   Memory Usage: {df.memory_usage(deep=True).sum() / 1024**2:.2f} MB")
    
    # Show sample data
    print(f"\nSample Data (First 3 Records):")
    print(df.head(3).to_string())
else:
    print("No data retrieved. Please check API connection.")


STEP 1: Extracting All Medicare Part D Data from CMS API
Fetching Medicare Part D data (target: 14,309 records)...
   API appears to limit responses to 5,000 records per request
Fetching Medicare Part D data (limit=5000, offset=0)...
Successfully fetched 5000 records
   Batch 1: 5,000 records | Total: 5,000
Fetching Medicare Part D data (limit=5000, offset=5000)...
Successfully fetched 5000 records
   Batch 2: 5,000 records | Total: 10,000
Fetching Medicare Part D data (limit=5000, offset=10000)...
Successfully fetched 4309 records
   Batch 3: 4,309 records | Total: 14,309
   Reached end of data (received 4,309 < 5,000)
SUCCESS: Fetched all 14,309 records!
   Expected: 14,309 | Actual: 14,309

Dataset Overview:
   Total Drug Records: 14,309
   Columns: ['Brnd_Name', 'Gnrc_Name', 'Tot_Mftr', 'Mftr_Name', 'Tot_Spndng_2019', 'Tot_Dsg_Unts_2019', 'Tot_Clms_2019', 'Tot_Benes_2019', 'Avg_Spnd_Per_Dsg_Unt_Wghtd_2019', 'Avg_Spnd_Per_Clm_2019', 'Avg_Spnd_Per_Bene_2019', 'Outlier_Flag_2019', 'T

In [4]:
# Cell 4: Clean Data and Load into SQL
def clean_data_for_sql(df):
    """Clean data and load into SQL database"""
    
    if df.empty:
        return None
    
    print("Cleaning data and setting up SQL database...")
    clean_df = df.copy()
    
    # Column mapping
    column_mapping = {
        'Brnd_Name': 'brand_name',
        'Gnrc_Name': 'generic_name',
        'Tot_Spndng_2023': 'total_spending_2023',
        'Tot_Clms_2023': 'total_claims_2023',
        'Tot_Benes_2023': 'total_beneficiaries_2023',
        'Avg_Spnd_Per_Clm_2023': 'avg_spending_per_claim_2023',
        'Avg_Spnd_Per_Dsg_Unt_Wghtd_2023': 'avg_spending_per_unit_2023',
        'Mftr_Name': 'manufacturer'
    }
    
    clean_df = clean_df.rename(columns=column_mapping)
    
    # Clean names and classify drug type
    clean_df['brand_name'] = clean_df['brand_name'].astype(str).str.strip().str.upper()
    clean_df['generic_name'] = clean_df['generic_name'].astype(str).str.strip().str.upper()
    clean_df['drug_type'] = clean_df.apply(lambda row: 'Generic' if row['brand_name'] == row['generic_name'] else 'Brand', axis=1)
    
    # Convert numeric columns
    numeric_columns = ['total_spending_2023', 'total_claims_2023', 'total_beneficiaries_2023',
                      'avg_spending_per_claim_2023', 'avg_spending_per_unit_2023']
    
    for col in numeric_columns:
        if col in clean_df.columns:
            clean_df[col] = pd.to_numeric(clean_df[col].astype(str).str.replace(r'[$,]', '', regex=True), errors='coerce')
    
    # Remove rows with missing critical data
    clean_df = clean_df.dropna(subset=['brand_name', 'generic_name'])
    
    # Create SQL database
    conn = sqlite3.connect(':memory:')
    clean_df.to_sql('medicare_drugs', conn, if_exists='replace', index=False)
    
    # Create indexes
    cursor = conn.cursor()
    cursor.execute("CREATE INDEX idx_drug_type ON medicare_drugs(drug_type)")
    cursor.execute("CREATE INDEX idx_generic_name ON medicare_drugs(generic_name)")
    cursor.execute("CREATE INDEX idx_brand_name ON medicare_drugs(brand_name)")
    
    print(f"SQL database created with {len(clean_df):,} records")
    return conn

# Clean data and create SQL database
sql_conn = clean_data_for_sql(df)

Cleaning data and setting up SQL database...
SQL database created with 14,309 records


In [5]:
# Cell 5: SQL Analysis Functions
def execute_sql_to_dataframe(conn, query, description=""):
    """Execute SQL and return DataFrame"""
    if description:
        print(f"\n{description}")
    
    try:
        result_df = pd.read_sql_query(query, conn)
        print(f"Query executed successfully. Returned {len(result_df)} rows.")
        return result_df
    except Exception as e:
        print(f"SQL Error: {e}")
        return pd.DataFrame()

def export_sql_result(conn, query, filename_prefix, description=""):
    """Execute SQL query and export to CSV"""
    result_df = execute_sql_to_dataframe(conn, query, description)
    
    if not result_df.empty:
        filename = f"powerbi_{filename_prefix}.csv"
        result_df.to_csv(filename, index=False)
        print(f"Exported: {filename} ({len(result_df):,} rows)")
        return filename, result_df
    
    return None, pd.DataFrame()

In [6]:
# Cell 6: Create All PowerBI Datasets Using SQL
def create_all_powerbi_datasets_sql(conn):
    """Create all PowerBI datasets using SQL queries"""
    
    exported_files = []
    
    print("Creating PowerBI datasets using SQL...")
    
    # 1. Main Drug Facts
    main_facts_sql = """
    SELECT 
        brand_name,
        generic_name,
        drug_type,
        manufacturer,
        total_spending_2023,
        total_claims_2023,
        total_beneficiaries_2023,
        avg_spending_per_claim_2023,
        avg_spending_per_unit_2023,
        CASE 
            WHEN total_beneficiaries_2023 > 0 
            THEN total_spending_2023 / total_beneficiaries_2023 
            ELSE NULL 
        END as cost_per_beneficiary_2023,
        CASE 
            WHEN total_beneficiaries_2023 > 0 
            THEN total_claims_2023 / total_beneficiaries_2023 
            ELSE NULL 
        END as claims_per_beneficiary_2023
    FROM medicare_drugs
    WHERE brand_name IS NOT NULL 
    AND generic_name IS NOT NULL
    """
    
    filename, df = export_sql_result(conn, main_facts_sql, "main_drug_facts", "Creating main drug facts table")
    if filename:
        exported_files.append(filename)
    
    # 2. Generic vs Brand Summary
    summary_sql = """
    SELECT 
        drug_type,
        COUNT(*) as drug_count,
        ROUND(100.0 * COUNT(*) / (SELECT COUNT(*) FROM medicare_drugs), 2) as percentage_of_drugs,
        COALESCE(SUM(total_spending_2023), 0) as total_spending,
        COALESCE(SUM(total_claims_2023), 0) as total_claims,
        COALESCE(SUM(total_beneficiaries_2023), 0) as total_beneficiaries,
        COALESCE(AVG(avg_spending_per_claim_2023), 0) as avg_cost_per_claim,
        COALESCE(AVG(avg_spending_per_unit_2023), 0) as avg_cost_per_unit,
        ROUND(100.0 * SUM(total_spending_2023) / (SELECT SUM(total_spending_2023) FROM medicare_drugs), 2) as percentage_of_spending
    FROM medicare_drugs
    WHERE drug_type IS NOT NULL
    GROUP BY drug_type
    ORDER BY total_spending DESC
    """
    
    filename, df = export_sql_result(conn, summary_sql, "generic_vs_brand_summary", "Creating generic vs brand summary")
    if filename:
        exported_files.append(filename)
    
    # 3. Savings Opportunities (THE MAIN ONE)
    savings_sql = """
    WITH drug_comparison AS (
        SELECT 
            generic_name,
            MAX(CASE WHEN drug_type = 'Brand' THEN avg_spending_per_claim_2023 END) as brand_cost_per_claim,
            MAX(CASE WHEN drug_type = 'Generic' THEN avg_spending_per_claim_2023 END) as generic_cost_per_claim,
            MAX(CASE WHEN drug_type = 'Brand' THEN total_claims_2023 END) as brand_claims,
            MAX(CASE WHEN drug_type = 'Generic' THEN total_claims_2023 END) as generic_claims,
            MAX(CASE WHEN drug_type = 'Brand' THEN total_spending_2023 END) as current_brand_spending,
            MAX(CASE WHEN drug_type = 'Brand' THEN total_beneficiaries_2023 END) as brand_beneficiaries
        FROM medicare_drugs
        WHERE drug_type IS NOT NULL 
        AND avg_spending_per_claim_2023 IS NOT NULL
        AND total_claims_2023 IS NOT NULL
        GROUP BY generic_name
        HAVING COUNT(DISTINCT drug_type) = 2
    )
    SELECT 
        generic_name,
        ROUND(brand_cost_per_claim, 2) as brand_cost_per_claim,
        ROUND(generic_cost_per_claim, 2) as generic_cost_per_claim,
        ROUND(brand_cost_per_claim - generic_cost_per_claim, 2) as savings_per_claim,
        ROUND(100.0 * (brand_cost_per_claim - generic_cost_per_claim) / brand_cost_per_claim, 2) as percent_savings,
        brand_claims,
        generic_claims,
        brand_beneficiaries,
        ROUND(current_brand_spending, 0) as current_brand_spending,
        ROUND(brand_claims * (brand_cost_per_claim - generic_cost_per_claim), 0) as total_potential_savings,
        ROUND(100.0 * generic_claims / (brand_claims + generic_claims), 2) as market_share_generic
    FROM drug_comparison
    WHERE brand_cost_per_claim > generic_cost_per_claim
    AND brand_cost_per_claim > 0
    AND generic_cost_per_claim > 0
    ORDER BY total_potential_savings DESC
    """
    
    filename, savings_df = export_sql_result(conn, savings_sql, "savings_opportunities", "Creating savings opportunities table")
    if filename:
        exported_files.append(filename)
    
    # 4. Top Opportunities with Impact Categories
    if not savings_df.empty:
        top_opportunities_sql = f"""
        WITH ranked_opportunities AS (
            {savings_sql.replace('ORDER BY total_potential_savings DESC', '')}
        ),
        top_50 AS (
            SELECT *,
            CASE 
                WHEN total_potential_savings >= 1000000000 THEN 'Critical Impact'
                WHEN total_potential_savings >= 400000000 THEN 'High Impact'
                WHEN total_potential_savings >= 50000000 THEN 'Medium Impact'
                ELSE 'Low Impact'
            END as impact_category
            FROM ranked_opportunities
            ORDER BY total_potential_savings DESC
            LIMIT 50
        )
        SELECT * FROM top_50
        """
        
        filename, df = export_sql_result(conn, top_opportunities_sql, "top_opportunities", "Creating top opportunities with impact categories")
        if filename:
            exported_files.append(filename)
    
    # 5. Manufacturer Analysis
    manufacturer_sql = """
    SELECT 
        manufacturer,
        drug_type,
        COUNT(*) as drug_count,
        ROUND(AVG(avg_spending_per_claim_2023), 2) as avg_cost_per_claim,
        ROUND(SUM(total_spending_2023), 0) as total_spending,
        ROUND(SUM(total_claims_2023), 0) as total_claims,
        ROUND(SUM(total_beneficiaries_2023), 0) as total_beneficiaries
    FROM medicare_drugs
    WHERE manufacturer IS NOT NULL 
    AND drug_type IS NOT NULL
    AND avg_spending_per_claim_2023 IS NOT NULL
    AND total_spending_2023 IS NOT NULL
    GROUP BY manufacturer, drug_type
    HAVING COUNT(*) >= 3
    ORDER BY total_spending DESC
    """
    
    filename, df = export_sql_result(conn, manufacturer_sql, "manufacturer_analysis", "Creating manufacturer analysis")
    if filename:
        exported_files.append(filename)
    
    return exported_files

# Generate all datasets
if sql_conn:
    exported_files = create_all_powerbi_datasets_sql(sql_conn)
    
    print(f"\nSUMMARY:")
    print(f"Generated {len(exported_files)} PowerBI-ready datasets:")
    for file in exported_files:
        print(f"  - {file}")

Creating PowerBI datasets using SQL...

Creating main drug facts table
Query executed successfully. Returned 14309 rows.
Exported: powerbi_main_drug_facts.csv (14,309 rows)

Creating generic vs brand summary
Query executed successfully. Returned 2 rows.
Exported: powerbi_generic_vs_brand_summary.csv (2 rows)

Creating savings opportunities table
Query executed successfully. Returned 322 rows.
Exported: powerbi_savings_opportunities.csv (322 rows)

Creating top opportunities with impact categories
Query executed successfully. Returned 50 rows.
Exported: powerbi_top_opportunities.csv (50 rows)

Creating manufacturer analysis
Query executed successfully. Returned 597 rows.
Exported: powerbi_manufacturer_analysis.csv (597 rows)

SUMMARY:
Generated 5 PowerBI-ready datasets:
  - powerbi_main_drug_facts.csv
  - powerbi_generic_vs_brand_summary.csv
  - powerbi_savings_opportunities.csv
  - powerbi_top_opportunities.csv
  - powerbi_manufacturer_analysis.csv


In [7]:
# Cell 7: Generate Summary Statistics
def generate_summary_stats(conn):
    """Generate the key summary statistics"""
    
    summary_query = """
    WITH drug_comparison AS (
        SELECT 
            generic_name,
            MAX(CASE WHEN drug_type = 'Brand' THEN avg_spending_per_claim_2023 END) as brand_cost_per_claim,
            MAX(CASE WHEN drug_type = 'Generic' THEN avg_spending_per_claim_2023 END) as generic_cost_per_claim,
            MAX(CASE WHEN drug_type = 'Brand' THEN total_claims_2023 END) as brand_claims,
            MAX(CASE WHEN drug_type = 'Brand' THEN total_spending_2023 END) as brand_spending
        FROM medicare_drugs
        WHERE drug_type IS NOT NULL 
        AND avg_spending_per_claim_2023 IS NOT NULL
        AND total_claims_2023 IS NOT NULL
        GROUP BY generic_name
        HAVING COUNT(DISTINCT drug_type) = 2
    )
    SELECT 
        COUNT(*) as drugs_with_savings_potential,
        SUM(brand_claims) as total_brand_claims,
        ROUND(SUM(brand_spending), 0) as total_current_brand_spending,
        ROUND(SUM(brand_claims * generic_cost_per_claim), 0) as cost_if_all_generic,
        ROUND(SUM(brand_claims * (brand_cost_per_claim - generic_cost_per_claim)), 0) as total_potential_savings,
        ROUND(100.0 * SUM(brand_claims * (brand_cost_per_claim - generic_cost_per_claim)) / SUM(brand_spending), 2) as percent_savings_potential
    FROM drug_comparison
    WHERE brand_cost_per_claim > generic_cost_per_claim
    AND brand_cost_per_claim > 0
    AND generic_cost_per_claim > 0
    """
    
    summary_df = execute_sql_to_dataframe(conn, summary_query, "Generating executive summary statistics")
    
    if not summary_df.empty:
        stats = summary_df.iloc[0]
        print(f"\nEXECUTIVE SUMMARY:")
        print(f"  Drugs with savings potential: {stats['drugs_with_savings_potential']:,}")
        print(f"  Total brand claims: {stats['total_brand_claims']:,}")
        print(f"  Current brand spending: ${stats['total_current_brand_spending']:,}")
        print(f"  Cost if all generic: ${stats['cost_if_all_generic']:,}")
        print(f"  TOTAL POTENTIAL SAVINGS: ${stats['total_potential_savings']:,}")
        print(f"  Percent savings potential: {stats['percent_savings_potential']:.1f}%")

if sql_conn:
    generate_summary_stats(sql_conn)


Generating executive summary statistics
Query executed successfully. Returned 1 rows.

EXECUTIVE SUMMARY:
  Drugs with savings potential: 322.0
  Total brand claims: 46,115,718.0
  Current brand spending: $17,175,111,432.0
  Cost if all generic: $35,316,483,211.0
  TOTAL POTENTIAL SAVINGS: $99,868,022,524.0
  Percent savings potential: 581.5%


In [8]:
# Cell 8: Cleanup
if sql_conn:
    sql_conn.close()
    print("Database connection closed")

print("\nANALYSIS COMPLETE!")
print("All PowerBI datasets generated using consistent SQL approach")

Database connection closed

ANALYSIS COMPLETE!
All PowerBI datasets generated using consistent SQL approach
